In [1]:
import pandas as pd
import torch


df_train = pd.read_excel(r'C:\Users\jites\Desktop\Project_folder\Jupyter_practice\diesease_train.xlsx')
df_test = pd.read_excel(r'C:\Users\jites\Desktop\Project_folder\Jupyter_practice\diesease_test.xlsx')

# df_train = df_train[:100]
# df_train = pd.concat([df_train.iloc[500:1500], df_train.iloc[2000:3000]])
# df_test = pd.concat([df_test.iloc[0:150], df_test.iloc[300:350]])
# df_test = df_test[100:500]

In [2]:
print(df_train.shape)
print(df_test.shape)

(4813, 2)
(1193, 2)


In [3]:
        ## check disease type of training data and test data are same or not
all_vals_train = [k for k,v in df_train['label'].value_counts().items()]
print(set(all_vals_train))

all_vals_test = [k for k,v in df_test['label'].value_counts().items()]
print(set(all_vals_test))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44}


In [4]:
# df_train['label'].value_counts()

In [5]:
df_train

,text,label
0,I have been having migraines and headaches. I ...,6
1,I have asthma and I get wheezing and breathing...,1
2,"cough,high_fever,breathlessness,family_history...",12
3,"chills,vomiting,high_fever,sweating,headache,n...",15
4,"I've been having back pain, a cough, and numbn...",0
...,...,...
4808,"itching,skin_rash,dischromic__patches",13
4809,"acidity,headache,blurred_and_distorted_vision,...",14
4810,"vomiting,breathlessness,sweating,chest_pain",40
4811,"vomiting,indigestion,loss_of_appetite,abdomina...",37


In [6]:
x_train = df_train['text']
y_train = df_train['label']
x_test = df_test['text']
y_test = df_test['label']

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = torch.tensor(label_encoder.fit_transform(y_train))
y_test_encoded = torch.tensor(label_encoder.transform(y_test))

In [8]:
# label_encoder.classes_

In [9]:
from transformers import BertTokenizer,BertForSequenceClassification, Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration 
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config, AdamW, DataCollatorForLanguageModeling, DataCollatorForSeq2Seq, DataCollatorWithPadding
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-custom-classification-tokenizer')
# tokenizer = LlamaTokenizer.from_pretrained('llama-7b')


# config = GPT2Config.from_pretrained('gpt2')
# config.pad_token_id = tokenizer.pad_token_id
# model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config, num_labels=len(label_encoder.classes_)).to(device)

# Load the pre-trained GPT-2 model and update configuration for classification
config = GPT2Config.from_pretrained('gpt2')
config.num_labels = len(label_encoder.classes_)  # Update num_labels based on your dataset

# model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config).to(device)
model = GPT2ForSequenceClassification.from_pretrained('./gpt2--custom-classification-model', config=config).to(device)
model.config.pad_token_id = model.config.eos_token_id
# model = GPT2ForSequenceClassification(configuration).from_pretrained('gpt-2).to(device)

# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_)).to(device)
# model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
# model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=len(label_encoder.classes_)).to(device)
# model = LlamaForSequenceClassification.from_pretrained('llama-7b', num_labels=len(label_encoder.classes_)).to(device)



model.config.attn_pdrop = 0.2
model.config.embd_pdrop = 0.2
model.config.resid_pdrop = 0.2


cuda


In [10]:
# !pip install tf-keras

In [11]:
def tokenize(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

tokenizer.pad_token = tokenizer.eos_token

train_encodings = tokenize(x_train.tolist())
test_encodings = tokenize(x_test.tolist())

In [12]:
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         # Use clone().detach() to convert data to tensors
#         item = {key: torch.tensor(val[idx]).clone().detach() for key, val in self.encodings.items()}
#         item['labels'] = self.labels[idx].clone().detach()  # Use 'labels' key for consistency
#         return item

#         ## warning
#     # def __getitem__(self, idx):
#     #     item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     #     item['label'] = self.labels[idx]
#     #     return item

#         ## this is for custome padding
#     # def __getitem__(self, idx):
#     #     item = {key: val[idx].tolist() for key, val in self.encodings.items()}  # Convert tensors to lists
#     #     item['labels'] = self.labels[idx].tolist()  # Convert tensor to list
#     #     return item

#     def __len__(self):
#         return len(self.labels)



### Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {}
        for key, val in self.encodings.items():
            if isinstance(val, torch.Tensor):
                item[key] = val[idx].clone().detach()
            else:
                item[key] = torch.tensor(val[idx])
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = CustomDataset(train_encodings, y_train_encoded)
test_dataset = CustomDataset(test_encodings, y_test_encoded)

In [13]:
# from torch.utils.data import DataLoader


# def pad_batch(batch):
#     max_length = max(len(x) for x in batch['input_ids'])
#     padded_batch = {
#         'input_ids': [x + [tokenizer.pad_token_id] * (max_length - len(x)) for x in batch['input_ids']],
#         'attention_mask': [([1] * len(x) + [0] * (max_length - len(x))) for x in batch['input_ids']]
#     }
#     return padded_batch



# def collate_fn(batch):
#     # Extract input_ids and attention_masks
#     input_ids = [item['input_ids'] for item in batch]
#     attention_masks = [item['attention_mask'] for item in batch]
    
#     # Apply padding
#     padded_batch = pad_batch({'input_ids': input_ids, 'attention_mask': attention_masks})
    
#     # Convert to tensors
#     input_ids_tensor = torch.tensor(padded_batch['input_ids'])
#     attention_mask_tensor = torch.tensor(padded_batch['attention_mask'])
#     labels_tensor = torch.tensor([item['labels'] for item in batch])

#     return {'input_ids': input_ids_tensor, 'attention_mask': attention_mask_tensor, 'labels': labels_tensor}



# train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn)


In [14]:

# # Initialize model and optimizer
# model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=len(label_encoder.classes_))
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # Training loop
# model.train()
# for batch in tqdm(train_loader):
#     input_ids = batch['input_ids']
#     attention_mask = batch['attention_mask']
#     labels = batch['labels']
    
#     optimizer.zero_grad()
    
#     outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#     loss = outputs.loss
#     loss.backward()
#     optimizer.step()

# # Evaluation loop
# model.eval()
# with torch.no_grad():
#     for batch in tqdm(test_loader):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']
        
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         # Compute metrics as needed

In [15]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import EarlyStoppingCallback

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)       ## weight_decay =>  L2 regularization
scheduler = ReduceLROnPlateau(optimizer, min_lr=7e-7)

In [16]:
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=False
# )


data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True
)


training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,      ## 2
    gradient_accumulation_steps=4,  # Accumulate over 4 batches to simulate a batch size of 32
    per_device_eval_batch_size=8,
    eval_steps=400,
    save_steps=500,
    warmup_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    # eval_strategy="steps",
    # save_strategy='steps',
    weight_decay=0.01,  ## L2 regularization
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,  # Load best model at the end based on evaluation
    metric_for_best_model='eval_loss',
    greater_is_better=False,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()


  0%|          | 0/450 [00:00<?, ?it/s]

c:\Users\jites\anaconda3\envs\torchenv\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.4856, 'grad_norm': 7.932785987854004, 'learning_rate': 5e-05, 'epoch': 0.66}


  0%|          | 0/150 [00:00<?, ?it/s]

{'eval_loss': 0.17114411294460297, 'eval_runtime': 327.4133, 'eval_samples_per_second': 3.644, 'eval_steps_per_second': 0.458, 'epoch': 1.0}
{'loss': 0.1875, 'grad_norm': 5.5900115966796875, 'learning_rate': 5e-05, 'epoch': 1.33}
{'loss': 0.1027, 'grad_norm': 2.436828136444092, 'learning_rate': 5e-05, 'epoch': 1.99}


  0%|          | 0/150 [00:00<?, ?it/s]

{'eval_loss': 0.07346154749393463, 'eval_runtime': 363.2819, 'eval_samples_per_second': 3.284, 'eval_steps_per_second': 0.413, 'epoch': 2.0}
{'loss': 0.0419, 'grad_norm': 0.0461387075483799, 'learning_rate': 5e-05, 'epoch': 2.66}


  0%|          | 0/150 [00:00<?, ?it/s]

{'eval_loss': 0.07262278348207474, 'eval_runtime': 356.4083, 'eval_samples_per_second': 3.347, 'eval_steps_per_second': 0.421, 'epoch': 2.99}
{'train_runtime': 22349.0042, 'train_samples_per_second': 0.646, 'train_steps_per_second': 0.02, 'train_loss': 0.18858757654825847, 'epoch': 2.99}


TrainOutput(global_step=450, training_loss=0.18858757654825847, metrics={'train_runtime': 22349.0042, 'train_samples_per_second': 0.646, 'train_steps_per_second': 0.02, 'total_flos': 3762565695406080.0, 'train_loss': 0.18858757654825847, 'epoch': 2.990033222591362})

In [17]:
# results = trainer.evaluate()
# print(results)

In [18]:
model.save_pretrained('./gpt2--custom-classification-model-3')
tokenizer.save_pretrained('./gpt2-custom-classification-tokenizer-3')

('./gpt2-custom-classification-tokenizer-3\\tokenizer_config.json',
 './gpt2-custom-classification-tokenizer-3\\special_tokens_map.json',
 './gpt2-custom-classification-tokenizer-3\\vocab.json',
 './gpt2-custom-classification-tokenizer-3\\merges.txt',
 './gpt2-custom-classification-tokenizer-3\\added_tokens.json')

In [1]:
from sklearn.preprocessing import LabelEncoder
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch
import json
from torch.nn.functional import softmax

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
label_encoder = LabelEncoder()
# def predict(texts):
#     encodings = tokenize(texts)
#     outputs = model(**encodings).to(device)
#     predictions = torch.argmax(outputs.logits, dim=1)
#     return label_encoder.inverse_transform(predictions.tolist())



tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-custom-classification-tokenizer-3')
model = GPT2ForSequenceClassification.from_pretrained('./gpt2--custom-classification-model-3').to(device)       ##, config=config

# def tokenize(texts):
#     return tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

def predict(texts):
    # Tokenize and move input tensors to the correct device
    # encodings = tokenize(texts)
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    
    # Ensure the model is on the correct device
    model.to(device)
    
    # Forward pass through the model
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Move outputs to CPU for processing
    logits = outputs.logits.cpu()
    
    # Get predictions
    predictions = torch.argmax(logits, dim=1)

    top_values, top_indices = torch.topk(logits, k=3)

    js_data = open(r'C:\Users\jites\Desktop\Project_folder\Jupyter_practice\diesease_data_reverse.json')
    json_data = json.load(js_data)

    predictions = json_data.get(str(int(predictions[-1])), 'Unknown disease')

    return predictions
    # return label_encoder.inverse_transform(predictions.tolist())
    #### return top_values, top_indices, logits


# Example prediction
# new_texts = ["fatigue, mood_swings, weight_loss, restlessness, sweating,diarrhoea, excessive_hunger, muscle_weakness, irritability, abnormal_menstruation"]
# new_texts = [" malaise, phlegm, throat_irritation, redness_of_eyes, sinus_pressure, runny_nose, congestion, chest_pain, loss_of_smell, muscle_pain"]
# new_texts = ["I have noticed cramps in my calves are becoming more frequent and intense. It is causing me a lot of discomforts. I am also overweight and my legs have started to swell. I am worried that I may have varicose veins."]
# new_texts = ["I have been feeling really sick lately. I have been itching all over, and I have been vomiting. I have also lost a lot of weight, and I have a fever. My skin has turned yellow, and my urine is dark. I have also been having abdominal pain."]
# new_texts = ["skin_rash, high_fever, blister, red_sore_around_nose, yellow_crust_ooze"]
new_texts = ["swelling in the right inguinal region associated with dragging pain x 2 years. Known hypertensive and on irregular treatment. Ho nocturnal frequency and on treatment. Problems vomiting,yellowish_skin,abdominal_pain,swelling_of_stomach,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload"]
predictions = predict(new_texts)


print("Input Symptoms => ")
print(new_texts[0])
# print("I have been feeling really sick lately. I have been itching all over, and I have been vomiting. I have also lost a lot of weight, and I have a fever.")
# print("My skin has turned yellow, and my urine is dark. I have also been having abdominal pain.")
print()
print("Output disease=> ", predictions)



# def get_probabilities(logits):
#     return softmax(logits, dim=-1)

# def calculate_top_k_accuracy(predictions, labels):
#     top_values, top_indices = get_top_k_predictions(predictions, k=k)
#     probabilities = get_probabilities(predictions)
    

#     js_data = open(r'C:\Users\jites\Desktop\Project_folder\Jupyter_practice\diesease_data_reverse.json')
#     json_data = json.load(js_data)


#     correct = 0
#     for i in range(len(labels)):
#         true_label = labels[i].item()
#         top_k_indices = top_indices[i].tolist()
#         top_k_probabilities = probabilities[i][top_k_indices].tolist()
        
#         if true_label in top_k_indices:
#             correct += 1
    
#     accuracy = correct / len(labels)
#     return top_values, top_indices, probabilities, accuracy







Input Symptoms => 
swelling in the right inguinal region associated with dragging pain x 2 years. Known hypertensive and on irregular treatment. Ho nocturnal frequency and on treatment. Problems vomiting,yellowish_skin,abdominal_pain,swelling_of_stomach,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload

Output disease=>  Alcoholic Hepatitis


c:\Users\jites\anaconda3\envs\torchenv\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
import torch

major, _ = torch.cuda.get_device_capability()
if major >= 8:
    print("=" * 80)
    print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
    print("=" * 80)

Your GPU supports bfloat16, you can accelerate training with the argument --bf16
